## Counterfactual Explanation on Structured Data
### CHAPTER 02 - *Model Explainability Methods*

From **Applied Machine Learning Explainability Techniques** by [**Aditya Bhattacharya**](https://www.linkedin.com/in/aditya-bhattacharya-b59155b6/), published by **Packt**

### Objective

In this notebook, we will try to implement some of the concepts related to Counterfactual explanations part of the Example based explainability methods discussed in Chapter 2 - Model Explainability Methods